In [1]:
import jieba
import numpy as np

def judgeodd(num):#判断否定词的个数，将否定词个数与2除进行求余数
    if (num % 2) == 0:
        return 'even'
    else:
        return 'odd'
    
path='C:/Users/bin/Desktop/电商综合案例/文本分析相关/情感分析/'
#定义函数，打开词典文件，返回列表
def open_dict(file_name = 'name', path=path):
    path = path + '%s.txt' % file_name
    dictionary = open(path, 'r', encoding='utf-8')
    dict = []
    for word in dictionary:
        word = word.strip('\n')
        dict.append(word)
    return dict

#注意，这里你要修改path路径。

deny_word = open_dict(file_name = '否定词', path= path)
posdict = open_dict(file_name = 'positive', path= path)
negdict = open_dict(file_name = 'negative', path= path)
degree_word = open_dict(file_name = '程度级别词语', path= path)

#定义情感词程度比例
mostdict = degree_word[degree_word.index('extreme')+1 : degree_word.index('very')]#权重4，即在情感词前乘以4
verydict = degree_word[degree_word.index('very')+1 : degree_word.index('more')]#权重3
moredict = degree_word[degree_word.index('more')+1 : degree_word.index('ish')]#权重2
ishdict = degree_word[degree_word.index('ish')+1 : degree_word.index('last')]#权重0.5

#开启情感分析
def sentiment_score_list(dataset):
    seg_sentence = dataset.split('。')
    count1=[]#为了记录积极与消极分的
    count2=[]#汇总count1
    for sen in seg_sentence: #循环遍历每一个评论
        i = 0 #记录扫描到的词的位置
        a = 0 #记录情感词的位置
        poscount = 0 #积极词的第一次分值
        poscdount2 = 0 #积极词反转后的分值
        poscount3=0 #积极词的最后得分
        negcount = 0 
        negcount2 = 0 
        negcount3 =0
        segtmp = jieba.lcut(sen, cut_all=False)  #把句子进行分词，以列表的形式返回
        for word in segtmp:
            if word in posdict:  # 判断词语是否是情感词
                poscount += 1
                c=0#记录否定词个数
                for w in segtmp[a:i]:  # 扫描情感词前的程度词
                    if w in mostdict:
                        poscount *= 4.0
                    elif w in verydict:
                        poscount *= 3.0
                    elif w in moredict:
                        poscount *= 2.0
                    elif w in ishdict:
                        poscount *= 0.5
                    # 计算否定词个数c
                    elif w in deny_word:
                        c += 1
                if judgeodd(c) == 'odd':  # 扫描情感词前的否定词数
                    poscount *= -1.0
                    poscount2 += poscount
                    poscount = 0
                    poscount3 = poscount + poscount2 + poscount3
                    poscount2 = 0
                else:
                    poscount3 = poscount + poscount2 + poscount3
                    poscount = 0
                a=i+1#记录情感词的位置变化
            
            elif word in negdict:# 消极情感的分析，与上面一致
                negcount +=1
                d = 0 #记录否定词
                for w in segtmp[a:i]:
                    if w in mostdict:
                        negcount *= 4.0
                    elif w in verydict:
                        negcount *= 3.0
                    elif w in moredict:
                        negcount *= 2.0
                    elif w in ishdict:
                        negcount *= 0.5
                    elif w in degree_word:
                        d += 1
                if judgeodd(d) == 'odd':
                    negcount *= -1.0
                    negcount2 += negcount
                    negcount = 0
                    negcount3 = negcount + negcount2 + negcount3
                    negcount2 = 0
                else:
                    negcount3 = negcount + negcount2 + negcount3
                    negcount = 0
                a = i + 1
            elif word =='!' or word =='！':#判断句子是否有感叹号
                for w2 in segtmp[::-1]:# 扫描感叹号前的情感词，发现后权值+2，然后退出循环
                    if w2 in posdict or negdict:
                        poscount3 += 2
                        negcount3 += 2
                        break
            i += 1 # 扫描词位置后移
            
            # 以下是防止出现负数的情况
            pos_count = 0
            neg_count = 0
            if poscount3 < 0 and negcount3 > 0:
                neg_count += negcount3 - poscount3
                pos_count = 0
            elif negcount3 < 0 and poscount3 > 0:
                pos_count = poscount3 - negcount3
                neg_count = 0
            elif poscount3 < 0 and negcount3 < 0:
                neg_count = -poscount3
                pos_count = -negcount3
            else:
                pos_count = poscount3
                neg_count = negcount3

            count1.append([pos_count, neg_count])
        count2.append(count1)
        count1 = []

    return count2
def sentiment_score(senti_score_list):
    score = []
    name=[]
    for review in senti_score_list:
        score_array = np.array(review)
        Pos = np.sum(score_array[:, 0])
        Neg = np.sum(score_array[:, 1])
        AvgPos = np.mean(score_array[:, 0])
        AvgPos = float('%.1f'%AvgPos)
        AvgNeg = np.mean(score_array[:, 1])
        AvgNeg = float('%.1f'%AvgNeg)
        StdPos = np.std(score_array[:, 0])
        StdPos = float('%.1f'%StdPos)
        StdNeg = np.std(score_array[:, 1])
        StdNeg = float('%.1f'%StdNeg)
        name.append(['积极分','消积分', '平均积极得分','平均消极得分','积极分标准差', '消积分标准差'])
        score.append([Pos,Neg,AvgPos,AvgNeg, StdPos,StdNeg])
    return score,name

In [2]:
data = '你就是个混蛋，混账玩意!产品烂到家！及其愤怒，非常生气，想骂人！！！！'

In [3]:
sentiment_score(sentiment_score_list(data))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\bin\AppData\Local\Temp\jieba.cache
Loading model cost 1.552 seconds.
Prefix dict has been built succesfully.


([[82.0, 168.0, 3.3, 6.7, 3.1, 5.8]],
 [['积极分', '消积分', '平均积极得分', '平均消极得分', '积极分标准差', '消积分标准差']])

In [5]:
data = '线材比较硬。听诊器效果严重。音质还可以。看来还是要去买个升级线。'

In [6]:
seg_sentence = data.split('。')

In [7]:
seg_sentence

['线材比较硬', '听诊器效果严重', '音质还可以', '看来还是要去买个升级线', '']

In [9]:
#线材比较硬

In [ ]:
        i = 0 #记录扫描到的词的位置
        a = 0 #记录情感词的位置
        poscount = 0 #积极词的第一次分值
        poscdount2 = 0 #积极词反转后的分值
        poscount3=0 #积极词的最后得分
        negcount = 0 
        negcount2 = 0 
        negcount3 =0

In [10]:
segtmp = jieba.lcut('线材比较硬', cut_all=False)

In [11]:
segtmp

['线材', '比较', '硬']

In [12]:
'线材' in posdict

False

In [13]:
'线材' in negdict

False

In [14]:
i = 1 #记录扫描到的词的位置
a = 0 #记录情感词的位置

In [15]:
'比较' in posdict

False

In [16]:
'比较' in negdict

False

In [ ]:
i = 2 #记录扫描到的词的位置
a = 0 #记录情感词的位置

In [17]:
'硬' in posdict

False

In [ ]:
'硬' in negdict  #true

In [ ]:
negcount3=3
1*3
negcount = 0
negcount2 = 0

In [ ]:
a=3
i=3

In [20]:
segtmp = jieba.lcut('听诊器效果严重', cut_all=False)

In [21]:
segtmp

['听诊器', '效果', '严重']

In [ ]:
'听诊器'
a=3
i=4

In [ ]:
'效果'
a=3
i=5

In [ ]:
'严重' 消极词中
negcount3=3+1=4
negcount = 0
negcount2 = 0
a=i+1=6
i=i+1=6